<a href="https://colab.research.google.com/github/zhe0/prac/blob/main/pyspark_clusteringOnNumericalAndVector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("ClusteringExample").getOrCreate()
spark

In [7]:
# df = spark.createDataFrame([
#     (1.0, Vectors.dense(3.0, 4.0), "a"),
#     (2.0, Vectors.dense(6.0, 7.0), "b"),
#     (3.0, Vectors.dense(9.0, 10.0), "c"),
#     (4.0, Vectors.dense(1.0, 2.0), "a"),
#     (5.0, Vectors.dense(4.0, 5.0), "b")
# ], ["feature1", "vector", "category"])

data = [
    (1.0, Vectors.dense(3.0, 4.0), "a"),
    (2.0, Vectors.dense(6.0, 7.0), "b"),
    (3.0, Vectors.dense(9.0, 10.0), "c"),
    (4.0, Vectors.dense(1.0, 2.0), "a"),
    (5.0, Vectors.dense(4.0, 5.0), "b")
]
df = spark.createDataFrame(data, ["feature1", "vector", "category"])

category_indexer = StringIndexer(inputCol="category", outputCol="category_index")

In [ ]:
assembler = VectorAssembler(
    inputCols=["feature1", "vector"],
    outputCol="features")

kmeans = KMeans(k=2, featuresCol="features")

pipeline = Pipeline(stages=[category_indexer, assembler, kmeans])

model = pipeline.fit(df)

predictions = model.transform(df)

In [13]:
predictions.show()

+--------+----------+--------+--------------+--------------+----------+
|feature1|    vector|category|category_index|      features|prediction|
+--------+----------+--------+--------------+--------------+----------+
|     1.0| [3.0,4.0]|       a|           0.0| [1.0,3.0,4.0]|         1|
|     2.0| [6.0,7.0]|       b|           1.0| [2.0,6.0,7.0]|         0|
|     3.0|[9.0,10.0]|       c|           2.0|[3.0,9.0,10.0]|         0|
|     4.0| [1.0,2.0]|       a|           0.0| [4.0,1.0,2.0]|         1|
|     5.0| [4.0,5.0]|       b|           1.0| [5.0,4.0,5.0]|         1|
+--------+----------+--------+--------------+--------------+----------+



In [14]:
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))
predictions.show()

Silhouette with squared euclidean distance = 0.6120640643838152
+--------+----------+--------+--------------+--------------+----------+
|feature1|    vector|category|category_index|      features|prediction|
+--------+----------+--------+--------------+--------------+----------+
|     1.0| [3.0,4.0]|       a|           0.0| [1.0,3.0,4.0]|         1|
|     2.0| [6.0,7.0]|       b|           1.0| [2.0,6.0,7.0]|         0|
|     3.0|[9.0,10.0]|       c|           2.0|[3.0,9.0,10.0]|         0|
|     4.0| [1.0,2.0]|       a|           0.0| [4.0,1.0,2.0]|         1|
|     5.0| [4.0,5.0]|       b|           1.0| [5.0,4.0,5.0]|         1|
+--------+----------+--------+--------------+--------------+----------+

